In [1]:
import pymysql
import pandas as pd
#這個cell是我為你們寫好的function
# 自行填入 host & passwd ， 在此處不提供
def Sent_Data_To_Mysql(name, article, keyword, table_name):
    db = pymysql.connect(host="YOUR HOST",port=3306, user="public_author",passwd="YOUR PASSWORD",db="AML_News" ,charset='utf8')
    content = db.escape_string(article)
    try:
        with db.cursor() as cursor:
            sql0 = "SELECT `id` FROM `"+table_name+"` WHERE '"+article+"' LIKE CONCAT('%', `article`, '%')"
            cursor.execute(sql0)
            duplicate_id = cursor.fetchone()
            if(duplicate_id is not None):#如果已經存在一個content重複的
                sql1 = " INSERT INTO `"+table_name+"` (`id`, `name`,`article`,`keyword`) VALUES (%s,%s,%s,%s) "
                sql2 = " ON DUPLICATE KEY UPDATE `article`= VALUES(`article`),`name`= VALUES(`name`),`keyword`= VALUES(`keyword`)"
                cursor.execute(sql1+sql2, (str(duplicate_id[0]), article,name,keyword))
                print(str(duplicate_id[0]))
            else:#如果沒有重複的
                sql = "INSERT INTO `"+table_name+"`(`article`,`name`,`keyword`) VALUES ( %s,%s,%s)"
                cursor.execute(sql, (article,name,keyword))
        db.commit()
    finally:
        db.close()

def Get_Data_From_Mysql(table_name,keyword):
    contents_list=[]
    target_ids =[]
    db = pymysql.connect(host="YOUR HOST",port=3306, user="public_author",passwd="YOUR PASSWORD",db="AML_News" ,charset='utf8')
    try:
        with db.cursor() as cursor:
            if(keyword=="all"):
                sql0 = "SELECT * FROM `"+table_name+"`"
                cursor.execute(sql0)
                contents_list = cursor.fetchall()
            else: 
                sql0 = "SELECT * FROM `"+table_name+"` WHERE '"+keyword+"' LIKE CONCAT('%', `keyword`)"
                cursor.execute(sql0)
                contents_list = cursor.fetchall()
                
        db.commit()
    finally:
        db.close()
    return  pd.DataFrame(list(contents_list), columns=['id','name','article','keyword'])

In [3]:
#大家共用的table名稱如下，這個資料庫是放人名的，法人的話名稱改為judical_criminal_org
table_name = 'judical_criminal_people'

keyword='背信'

output_dataframe = Get_Data_From_Mysql(table_name,keyword)
print(len(output_dataframe))#0代表裡面沒有內容

#output_dataframe = Get_Data_From_Mysql(source_name,keyword)
#output_dataframe.head()

0


In [6]:
import time
import re
from datetime import datetime
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import json
import requests
import pandas as pd

crime_keyword = "洗錢"#更改這個，關鍵字找到結果500筆資料

#檢查是否為最後一頁，輸入keyword後檢索結果頁面右上角會有第幾頁，共幾頁的資料
def check_final_page(s):
    a = s.split('現在第')
    a = a[1].split('/')
    now = a[0].strip()
    total = a[1].split(' 頁')[0].strip()
    if(now=='25'): return True
    return (now==total)


main_driver = webdriver.Chrome('C:\\Users\\user\\Desktop\\chromedriver')  # 注意你們放CHROMEDRIVER的位置
main_driver.implicitly_wait(10)
main_driver.get('https://law.judicial.gov.tw/FJUD/defaulte.aspx')

#簡單檢所的頁面中，輸入關鍵字，這邊你們可以放入洗錢嫌疑程度的關鍵字
keyword_input_ele = main_driver.find_element_by_xpath('//*[@id="txtKW"]')
keyword_input_ele.send_keys(crime_keyword)

btn_ele = keyword_input_ele.find_element_by_xpath('//*[@id="btnSimpleQry"]')
btn_ele.click()
#這邊很重要，要切換到iframe內
main_driver.switch_to.frame("iframe-data")
case_link_list=[]

while True:
    
    ##取的一頁中的所有檢索案件
    case_list = main_driver.find_elements_by_id('hlTitle')
    for idx, case in enumerate(case_list):
        case_link_list.append(case.get_attribute('href'))#將每個案件的檢索結果頁面連結放在case_link_list中
    #注意因為網頁最多只會提供500比檢索資料，因此case_link_list長度最多500
    #如果是最後一頁，就跳出
    if(not check_final_page(main_driver.find_element_by_xpath('//*[@id="plPager"]/span[1]').text)):
        next_page_btn_ele = main_driver.find_element_by_id('hlNext')
        next_page_btn_ele.click()  
    else: break
main_driver.close()

In [7]:
case_link_list

['https://law.judicial.gov.tw/FJUD/data.aspx?ro=0&q=7b10b8962980857cf40213347410eee4&sort=DS&ot=in',
 'https://law.judicial.gov.tw/FJUD/data.aspx?ro=1&q=7b10b8962980857cf40213347410eee4&sort=DS&ot=in',
 'https://law.judicial.gov.tw/FJUD/data.aspx?ro=2&q=7b10b8962980857cf40213347410eee4&sort=DS&ot=in',
 'https://law.judicial.gov.tw/FJUD/data.aspx?ro=3&q=7b10b8962980857cf40213347410eee4&sort=DS&ot=in',
 'https://law.judicial.gov.tw/FJUD/data.aspx?ro=4&q=7b10b8962980857cf40213347410eee4&sort=DS&ot=in',
 'https://law.judicial.gov.tw/FJUD/data.aspx?ro=5&q=7b10b8962980857cf40213347410eee4&sort=DS&ot=in',
 'https://law.judicial.gov.tw/FJUD/data.aspx?ro=6&q=7b10b8962980857cf40213347410eee4&sort=DS&ot=in',
 'https://law.judicial.gov.tw/FJUD/data.aspx?ro=7&q=7b10b8962980857cf40213347410eee4&sort=DS&ot=in',
 'https://law.judicial.gov.tw/FJUD/data.aspx?ro=8&q=7b10b8962980857cf40213347410eee4&sort=DS&ot=in',
 'https://law.judicial.gov.tw/FJUD/data.aspx?ro=9&q=7b10b8962980857cf40213347410eee4&sort=D

In [11]:
import pandas as pd
df = pd.DataFrame(columns=['名稱','正文'])
nonsense_char_list=['○','.','律師','○','○○','']
start_targets_list = ["被　　　告","即被告　","上訴人即被告　"]

def Check_Noncense(line_string):#注意在第12行，謝○○的人名這樣的就可以用此方法去掉
    line_string = line_string.strip()
    for nonsense_char in nonsense_char_list:
        if(len(line_string)<4 and line_string.find(nonsense_char)!= -1): return True#大於四個字的有可能會有相關資訊
    return False    

#注意在第11行，找到有"被　　　告"的關鍵字之後就開始搜尋這個項目裡的名子，直到
#第12行的「謝○○」為止
def Check_Start_Target_Title(line_string): 
    output=''
    for start_target in start_targets_list:
        if(line_string.find(start_target)!= -1):            
            output = line_string.replace(start_target,"")
            return output, True
    return output, False

#做把各篇判決書內的被告取出名子的方法
def Get_Name_List(context):
    start_checking_mode = False
    temp = pd.DataFrame(columns=['名稱','正文'])
    #先把判決書的每行分開
    lines = context.split('\n')
    #接著對判決書內的每行做處理   
    for line in lines:
        if(start_checking_mode): 
            #步驟3，接著找下一行是否有同樣屬於被告等類似的title範圍內的名子，這裡參考第12行，前面會有很長的空格
            if(line.find("　　　　　")==0): #步驟4，如果有且Check_Noncense發現裡面不是存在無法辨識的名子，則放入df中
                if(Check_Noncense(line) == False): temp = temp.append({'名稱':line.strip(),'正文':context}, ignore_index=True)            
            else: start_checking_mode = False #步驟5，如果在步驟4沒發現，則繼續望下一行找(跳回地62行)
        else:
            #步驟1，先確定是否有被告等類似的title出現，參考第11行範例，如果有出現，則start_checking_mode=true
            #如果沒有，則繼續找看看下一行否有被告等類似的title出現
            start_name, start_checking_mode  =  Check_Start_Target_Title(line)
            if(start_checking_mode) : 
                #步驟2，如果找到，則先把該行名子取出來，這裡會取出第11行的"九福國際人力仲介有限公司"
                if(Check_Noncense(start_name) == False): temp = temp.append({'名稱':start_name,'正文':context}, ignore_index=True)
    return temp


main_driver = webdriver.Chrome('C:\\Users\\user\\Desktop\\chromedriver')  # 注意你們放CHROMEDRIVER的位置
main_driver.implicitly_wait(10)
for case_link in case_link_list:
    if(case_link==None): continue 
    #----------------這邊從每篇判決書文字中取得我們要的人名------------------#
    main_driver.get(case_link)
    case_text = main_driver.find_element_by_class_name('text-pre').text
    df = df.append(Get_Name_List(case_text))
    time.sleep(1)
    #-------------------------------------------------------------------#
main_driver.close()
#變成dataframe方便之後處理，接下來就是上傳到資料庫啦~
df = df.reset_index(drop=True)#把index重新整理
df.head(10)

,名稱,正文
0,李慧慧,臺灣新竹地方法院刑事簡易判決 108年度金簡字第21號\n聲 請 人 臺灣新竹地方檢...
1,王菏勵,臺灣新竹地方法院刑事簡易判決 107年度金簡字第30號\n聲 請 人 臺灣新竹地方檢...
2,余恩中,臺灣苗栗地方法院刑事裁定 108年度金訴字第8號\n公 訴 人 臺灣苗栗地方檢察...
3,楊光正,臺灣新竹地方法院刑事簡易判決 108年度竹簡字第129號\n聲 請 人 臺灣新竹地方檢...
4,黃彥銘,臺灣新竹地方法院刑事判決 107年度訴字第868號\n公 訴 人 臺灣新竹地方檢...
5,林冠,臺灣新竹地方法院刑事簡易判決 108年度金簡字第7號\n聲 請 人 臺灣新竹地方檢察...
6,彭功明,臺灣苗栗地方法院刑事判決 107年度金訴字第1號\n公 訴 人 臺灣苗栗地方檢察...
7,林緯隆,臺灣苗栗地方法院刑事判決 107年度原訴字第33號\n公 訴 人 臺灣苗栗地方檢...
8,楊凱翔,臺灣基隆地方法院刑事判決 108年度金簡上字第1號\n上 訴 人 臺灣基隆地方檢察...
9,程彥傑,臺灣新竹地方法院刑事簡易判決\n 108年度竹東簡字第58...


In [12]:
org_filter_list = ['公司','委員會','基金會']
df_org = pd.DataFrame(columns=['法人名稱','判決正文','犯罪類別'])
df_people = pd.DataFrame(columns=['人名','判決正文','犯罪類別'])

#將公司跟人名分開
df_org = df[df['名稱'].str.contains('|'.join(org_filter_list))]
df_people = df[~df['名稱'].str.contains('|'.join(org_filter_list))]

In [13]:
df_org.head()

,名稱,正文
46,燈興科技事業股份有限公司,臺灣新竹地方法院民事判決 108年度重訴字第18號\n原 告 金麗科技股份有...
322,華南商業銀行股份有限公司基隆分公司,臺灣基隆地方法院基隆簡易庭民事判決\n 107年度基簡字第...
537,熊熊租賃有限公司,宣 示 判 決 筆 錄 106年度板簡字第2801號\n原 告 李冠陞\n被...
579,IRONYUN INCORPORATED台灣分公司,臺灣新竹地方法院民事簡易判決\n 106年度竹北簡字第311...


In [14]:
df_people.head()

,名稱,正文
0,李慧慧,臺灣新竹地方法院刑事簡易判決 108年度金簡字第21號\n聲 請 人 臺灣新竹地方檢...
1,王菏勵,臺灣新竹地方法院刑事簡易判決 107年度金簡字第30號\n聲 請 人 臺灣新竹地方檢...
2,余恩中,臺灣苗栗地方法院刑事裁定 108年度金訴字第8號\n公 訴 人 臺灣苗栗地方檢察...
3,楊光正,臺灣新竹地方法院刑事簡易判決 108年度竹簡字第129號\n聲 請 人 臺灣新竹地方檢...
4,黃彥銘,臺灣新竹地方法院刑事判決 107年度訴字第868號\n公 訴 人 臺灣新竹地方檢...


In [15]:
for index, row in df_people.iterrows():
    #print(row['人名'])
    
    #格式參考Sent_Data_To_Mysql(name, article, keyword, table_name)
    
    #Sent_Data_To_Mysql(row['名稱'],row['正文'],crime_keyword,table_name)#法人用法跟人名一樣
    
    #注意['正文']跟['名稱']內不要有''這種符號出現~不然就會跟助教一樣mysql SQL syntax上面有錯誤喔!
    Sent_Data_To_Mysql(row['名稱'],row['正文'],crime_keyword,table_name)

14
14


C:\Users\user\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'article' at row 1")
  result = self._query(query)


23
23
26
26
26
26
26


C:\Users\user\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'name' at row 1")
  result = self._query(query)


84
105
116
116
116
156
156
156
156
156
161
156
156
156
170
204


ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 's will take 3 days then with │RP：所以大概會花三天，如果你在那' at line 1")

In [43]:
df_people.iloc[[204]]

,名稱,正文
205,陳凱威,臺灣新竹地方法院刑事簡易判決\n 107年度竹東簡字第279...
